In [4]:
import itertools

class PropositionalLogic:
    def __init__(self):
        self.symbols = set()

    def extract_symbols(self, expression):
        """Extract propositional symbols from logical expression"""
        # Remove operators and parentheses, then split
        operators = {'→', '¬', '∧', '∨', '(', ')', ' ', '~', '!', '&', '|', '->'}
        current_symbol = ''

        for char in expression:
            if char not in operators:
                current_symbol += char
            else:
                if current_symbol:
                    self.symbols.add(current_symbol)
                    current_symbol = ''

        if current_symbol:
            self.symbols.add(current_symbol)

    def parse_expression(self, expression, model):
        """Parse and evaluate a logical expression given a model"""
        # Convert to standard format for easier parsing
        expr = expression.replace('→', '->').replace('¬', '!').replace('∧', '&').replace('∨', '|')

        # Tokenize the expression
        tokens = []
        current = ''

        for char in expr:
            if char in '()!&|-> ':
                if current:
                    tokens.append(current)
                    current = ''
                if char != ' ':
                    tokens.append(char)
            else:
                current += char

        if current:
            tokens.append(current)

        # Evaluate the expression
        return self.evaluate(tokens, model)

    def evaluate(self, tokens, model):
        """Recursively evaluate tokens"""
        if not tokens:
            return True

        # Handle parentheses
        if tokens[0] == '(':
            # Find matching closing parenthesis
            count = 1
            for i in range(1, len(tokens)):
                if tokens[i] == '(':
                    count += 1
                elif tokens[i] == ')':
                    count -= 1
                if count == 0:
                    inner_result = self.evaluate(tokens[1:i], model)
                    remaining_result = self.evaluate(tokens[i+1:], model)
                    return inner_result and remaining_result

        # Handle negation
        if tokens[0] == '!':
            return not self.evaluate(tokens[1:], model)

        # Handle binary operators
        for i, token in enumerate(tokens):
            if token == '->':
                left = self.evaluate(tokens[:i], model)
                right = self.evaluate(tokens[i+1:], model)
                return (not left) or right  # Implication: p->q ≡ ¬p ∨ q

            elif token == '&':
                left = self.evaluate(tokens[:i], model)
                right = self.evaluate(tokens[i+1:], model)
                return left and right

            elif token == '|':
                left = self.evaluate(tokens[:i], model)
                right = self.evaluate(tokens[i+1:], model)
                return left or right

        # Base case: propositional symbol
        if tokens[0] in model:
            return model[tokens[0]]

        # If we reach here, it's an unrecognized token
        return False

    def generate_truth_assignments(self):
        """Generate all possible truth assignments for the symbols"""
        symbols_list = sorted(list(self.symbols))
        assignments = []

        # Generate all combinations of True/False for each symbol
        for values in itertools.product([True, False], repeat=len(symbols_list)):
            assignment = {}
            for i, symbol in enumerate(symbols_list):
                assignment[symbol] = values[i]
            assignments.append(assignment)

        return assignments

    def entails(self, kb, query):
        """Check if KB entails query using truth-table enumeration"""
        # Extract all symbols from KB and query
        self.symbols = set()
        for sentence in kb:
            self.extract_symbols(sentence)
        self.extract_symbols(query)

        # Generate all possible models
        models = self.generate_truth_assignments()

        print("Truth Table:")
        print("=" * 60)

        # Print header
        header = " | ".join(sorted(self.symbols))
        kb_header = " | ".join([f"KB{i+1}" for i in range(len(kb))])
        print(f"{header} | {kb_header} | KB | Query")
        print("-" * 60)

        kb_models = []  # Models where KB is true

        for model in models:
            # Evaluate each KB sentence
            kb_results = []
            kb_true = True

            for sentence in kb:
                result = self.parse_expression(sentence, model)
                kb_results.append(result)
                kb_true = kb_true and result

            # Evaluate query
            query_result = self.parse_expression(query, model)

            # Print row
            row = " | ".join(["T" if model[sym] else "F" for sym in sorted(self.symbols)])
            kb_row = " | ".join(["T" if r else "F" for r in kb_results])
            print(f"{row} | {kb_row} | {'T' if kb_true else 'F'} | {'T' if query_result else 'F'}")

            if kb_true:
                kb_models.append(model)

        print("=" * 60)

        # Check entailment: KB ⊨ query iff in every model where KB is true, query is also true
        entailment = True
        for model in kb_models:
            if not self.parse_expression(query, model):
                entailment = False
                break

        return entailment, kb_models

def main():
    pl = PropositionalLogic()

    # Problem from the observation book
    print("PROBLEM: Knowledge Base Entailment")
    print("Knowledge Base:")
    kb = ["Q → P", "P → ¬Q", "Q ∨ R"]
    print("1. Q → P")
    print("2. P → ¬Q")
    print("3. Q ∨ R")
    print()

    # Test each query
    queries = ["R", "R → P", "Q → R"]

    for query in queries:
        print(f"\n{'='*50}")
        print(f"Query: Does KB entail {query}?")
        print(f"{'='*50}")

        entails_result, kb_models = pl.entails(kb, query)

        print(f"\nResult: KB {'⊨' if entails_result else '⊭'} {query}")
        print(f"Number of models where KB is true: {len(kb_models)}")

        if entails_result:
            print(f"✓ In ALL models where KB is true, {query} is also true")
        else:
            print(f"✗ There exists a model where KB is true but {query} is false")

if __name__ == "__main__":
    main()

PROBLEM: Knowledge Base Entailment
Knowledge Base:
1. Q → P
2. P → ¬Q
3. Q ∨ R


Query: Does KB entail R?
Truth Table:
P | Q | R | KB1 | KB2 | KB3 | KB | Query
------------------------------------------------------------
T | T | T | T | T | T | T | T
T | T | F | T | T | T | T | F
T | F | T | F | T | T | F | T
T | F | F | F | T | F | F | F
F | T | T | T | F | T | F | T
F | T | F | T | F | T | F | F
F | F | T | F | F | T | F | T
F | F | F | F | F | F | F | F

Result: KB ⊭ R
Number of models where KB is true: 2
✗ There exists a model where KB is true but R is false

Query: Does KB entail R → P?
Truth Table:
P | Q | R | KB1 | KB2 | KB3 | KB | Query
------------------------------------------------------------
T | T | T | T | T | T | T | T
T | T | F | T | T | T | T | F
T | F | T | F | T | T | F | T
T | F | F | F | T | F | F | F
F | T | T | T | F | T | F | T
F | T | F | T | F | T | F | F
F | F | T | F | F | T | F | T
F | F | F | F | F | F | F | F

Result: KB ⊭ R → P
Number of models where KB 

In [2]:
import itertools

class PropositionalLogic:
    def __init__(self):
        self.symbols = set()

    def extract_symbols(self, expression):
        """Extract propositional symbols from logical expression"""
        operators = {'→', '¬', '∧', '∨', '(', ')', ' ', '~', '!', '&', '|', '->'}
        current_symbol = ''

        for char in expression:
            if char not in operators:
                current_symbol += char
            else:
                if current_symbol:
                    self.symbols.add(current_symbol)
                    current_symbol = ''

        if current_symbol:
            self.symbols.add(current_symbol)

    def evaluate_expression(self, expression, model):
        """Evaluate a logical expression given a model"""
        expr = expression.replace('→', '->').replace('¬', '!')

        # Handle implication: P → Q ≡ ¬P ∨ Q
        if '->' in expr:
            parts = expr.split('->')
            left = self.evaluate_expression(parts[0].strip(), model)
            right = self.evaluate_expression(parts[1].strip(), model)
            return (not left) or right

        # Handle negation
        if expr.startswith('!'):
            return not self.evaluate_expression(expr[1:].strip(), model)

        # Handle OR
        if '∨' in expr:
            parts = expr.split('∨')
            left = self.evaluate_expression(parts[0].strip(), model)
            right = self.evaluate_expression(parts[1].strip(), model)
            return left or right

        # Handle AND
        if '∧' in expr:
            parts = expr.split('∧')
            left = self.evaluate_expression(parts[0].strip(), model)
            right = self.evaluate_expression(parts[1].strip(), model)
            return left and right

        # Base case: propositional symbol
        return model.get(expr.strip(), False)

    def generate_truth_table(self, kb, queries):
        """Generate complete truth table for KB and check entailment for queries"""
        # Extract all symbols
        self.symbols = set()
        for sentence in kb:
            self.extract_symbols(sentence)
        for query in queries:
            self.extract_symbols(query)

        symbols_list = sorted(list(self.symbols))
        models = []

        # Generate all truth assignments in STANDARD ORDER (P changes slowest)
        # For 3 variables: P, Q, R - P alternates every 4 rows
        for values in itertools.product([True, False], repeat=len(symbols_list)):
            model = {}
            for i, symbol in enumerate(symbols_list):
                model[symbol] = values[i]
            models.append(model)

        # REORDER models to have standard truth table order
        # P should change slowest, then Q, then R fastest
        if symbols_list == ['P', 'Q', 'R']:
            # Reorder to standard truth table pattern
            reordered_models = []
            # P = True cases (first 4 rows)
            reordered_models.append({'P': True, 'Q': True, 'R': True})
            reordered_models.append({'P': True, 'Q': True, 'R': False})
            reordered_models.append({'P': True, 'Q': False, 'R': True})
            reordered_models.append({'P': True, 'Q': False, 'R': False})
            # P = False cases (last 4 rows)
            reordered_models.append({'P': False, 'Q': True, 'R': True})
            reordered_models.append({'P': False, 'Q': True, 'R': False})
            reordered_models.append({'P': False, 'Q': False, 'R': True})
            reordered_models.append({'P': False, 'Q': False, 'R': False})
            models = reordered_models

        # Print header
        print("TRUTH TABLE (Standard Order)")
        print("=" * 80)
        header = " | ".join(symbols_list)
        kb_header = " | ".join([f"KB{i+1}" for i in range(len(kb))])
        query_header = " | ".join([f"Q{i+1}" for i in range(len(queries))])
        print(f"{header} | {kb_header} | KB  | {query_header}")
        print("-" * 80)

        kb_models = []  # Models where KB is true

        for model in models:
            # Evaluate KB sentences
            kb_results = []
            for sentence in kb:
                result = self.evaluate_expression(sentence, model)
                kb_results.append(result)

            kb_true = all(kb_results)  # KB is true only if ALL sentences are true

            # Evaluate queries
            query_results = []
            for query in queries:
                result = self.evaluate_expression(query, model)
                query_results.append(result)

            # Print row
            symbol_values = " | ".join(["T" if model[sym] else "F" for sym in symbols_list])
            kb_values = " | ".join(["T" if r else "F" for r in kb_results])
            query_values = " | ".join(["T" if r else "F" for r in query_results])
            print(f"{symbol_values} | {kb_values} | {'T ' if kb_true else 'F '} | {query_values}")

            if kb_true:
                kb_models.append(model)

        print("=" * 80)
        return kb_models, models

def solve_problem():
    pl = PropositionalLogic()

    # Given problem
    kb = ["Q → P", "P → ¬Q", "Q ∨ R"]
    queries = ["R", "R → P", "Q → R"]

    print("KNOWLEDGE BASE:")
    for i, sentence in enumerate(kb, 1):
        print(f"KB{i}: {sentence}")
    print()

    print("QUERIES:")
    for i, query in enumerate(queries, 1):
        print(f"Q{i}: {query}")
    print()

    # Generate truth table
    kb_models, all_models = pl.generate_truth_table(kb, queries)

    print(f"\nMODELS WHERE KB IS TRUE: {len(kb_models)}")
    for i, model in enumerate(kb_models, 1):
        print(f"Model {i}: {model}")

    print("\nENTAILMENT RESULTS:")
    print("=" * 50)

    for i, query in enumerate(queries, 1):
        print(f"\nQuery Q{i}: {query}")

        # KB entails query if in EVERY model where KB is true, query is also true
        entails = True
        for model in kb_models:
            query_result = pl.evaluate_expression(query, model)
            if not query_result:
                entails = False
                break

        if entails:
            print(f"✓ KB ⊨ {query} (ENTAILS)")
            print(f"  Reasoning: In all {len(kb_models)} models where KB is true, {query} is also true")
        else:
            print(f"✗ KB ⊭ {query} (DOES NOT ENTAIL)")
            # Find counterexample
            for model in kb_models:
                query_result = pl.evaluate_expression(query, model)
                if not query_result:
                    print(f"  Counterexample: In model {model}, KB is true but {query} is false")
                    break

# Run the solution
if __name__ == "__main__":
    solve_problem()

KNOWLEDGE BASE:
KB1: Q → P
KB2: P → ¬Q
KB3: Q ∨ R

QUERIES:
Q1: R
Q2: R → P
Q3: Q → R

TRUTH TABLE (Standard Order)
P | Q | R | KB1 | KB2 | KB3 | KB  | Q1 | Q2 | Q3
--------------------------------------------------------------------------------
T | T | T | T | F | T | F  | T | T | T
T | T | F | T | F | T | F  | F | T | F
T | F | T | T | T | T | T  | T | T | T
T | F | F | T | T | F | F  | F | T | T
F | T | T | F | T | T | F  | T | F | T
F | T | F | F | T | T | F  | F | T | F
F | F | T | T | T | T | T  | T | F | T
F | F | F | T | T | F | F  | F | T | T

MODELS WHERE KB IS TRUE: 2
Model 1: {'P': True, 'Q': False, 'R': True}
Model 2: {'P': False, 'Q': False, 'R': True}

ENTAILMENT RESULTS:

Query Q1: R
✓ KB ⊨ R (ENTAILS)
  Reasoning: In all 2 models where KB is true, R is also true

Query Q2: R → P
✗ KB ⊭ R → P (DOES NOT ENTAIL)
  Counterexample: In model {'P': False, 'Q': False, 'R': True}, KB is true but R → P is false

Query Q3: Q → R
✓ KB ⊨ Q → R (ENTAILS)
  Reasoning: In all 2 models